In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('wagers.csv')

# Display the first few rowss to verify
print("Data Preview:")
df

Data Preview:


,Player,Side,Wager
0,Adi,Subhajeet,100
1,Sai,Harsh,120
2,Kevin,Subhajeet,150


In [30]:
sides = df['Side'].unique()
sides

array(['Subhajeet', 'Harsh'], dtype=object)

In [23]:
totals = df.groupby('Side')['Wager'].sum()
total_A = totals.get(sides[0], 0)
total_B = totals.get(sides[1], 0)

print(f"\nTotal wager on {sides[0]}: {total_A}")
print(f"Total wager on {sides[1]}: {total_B}")


Total wager on Subhajeet: 250
Total wager on Harsh: 120


In [25]:
def calculate_payout(row):
    side = row['Side']
    wager = row['Wager']
    
    # If Side A wins, players on Side A get a share of total wagered on Side B
    if side == sides[0]:
        return wager / total_A * total_B if total_A > 0 else 0
    # If Side B wins, players on Side B get a share of total wagered on Side A
    elif side == sides[1]:
        return wager / total_B * total_A if total_B > 0 else 0
    else:
        return 0

In [26]:
df[f'Payout_if_{sides[0]}_wins'] = df.apply(lambda row: calculate_payout(row) if row['Side'] == sides[0] else 0, axis=1)
df[f'Payout_if_{sides[1]}_wins'] = df.apply(lambda row: calculate_payout(row) if row['Side'] == sides[1] else 0, axis=1)

In [28]:
df

,Player,Side,Wager,Payout_if_Subhajeet_wins,Payout_if_Harsh_wins
0,Adi,Subhajeet,100,48.0,0.0
1,Sai,Harsh,120,0.0,250.0
2,Kevin,Subhajeet,150,72.0,0.0
